In [2]:
import sys; sys.path.insert(0, "..")
from multifunbrain import *
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.cluster.hierarchy import dendrogram, fcluster
import pickle

In [4]:
path_root = Path.cwd().parent
path_data = path_root / Path("data")
path_atlas = Path('C:\\Users\\margh\\OneDrive - Politecnico di Milano\\Desktop\\multifun-brain\\atlas_timecourses')
os.chdir(path_root)
print(os.getcwd())

results_dir = path_root / "results"
results_dir.mkdir(exist_ok=True)

# Temporal resolution 
fs_BOLD = 1/1.353
fs_BOLD_rs = 1/0.98
fs_VASO = 1/1.353
fs_ASL = 1/5.5


# Define frequency bands
slow5_min = 0.01
slow5_max = 0.027
slow4_min = 0.027
slow4_max = 0.073

f_test_min = 0.073
f_test_max = 0.18



c:\Users\margh\OneDrive - Politecnico di Milano\Desktop


In [5]:
signal_files = [
    "HarvardOxford_48Parcels_kwCBF4D.ts.1D",
    "HarvardOxford_48Parcels_kwfcurN_Vaso.ts.1D",
    "HarvardOxford_48Parcels_kwfurN_Bold.ts.1D",
    "HarvardOxford_48Parcels_kwoptcomMIRDenoised_bold.ts.1D"
]

signal_names = [
    "ASL",
    "VASO",
    "BOLD",
    "BOLD_rs"
]

subject_folders = [f for f in path_atlas.iterdir() if f.is_dir() and f.name.startswith("sub-")]


In [9]:
correlation_matrices = {
    signal_name: {
        "slow4": [],
        "slow5": [],
        "f_test": []
    } for signal_name in signal_names
}

for subject_folder in subject_folders:
    subject_id = subject_folder.name
    
    subject_results_dir = results_dir / subject_id
    
    for idx, (signal_file, signal_name) in enumerate(zip(signal_files, signal_names)):
        file_path = subject_folder / signal_file
        
        if file_path.exists():

            if signal_name == 'BOLD':
                fs = fs_BOLD
            elif signal_name == 'BOLD_rs':
                fs = fs_BOLD_rs
            elif signal_name == 'ASL':
                fs = fs_ASL
            elif signal_name == 'VASO':
                fs = fs_VASO
            
            ts_original = np.loadtxt(file_path)
            num_timepoints, num_rois = ts_original.shape
            
            ts_slow5 = np.zeros_like(ts_original)
            ts_slow4 = np.zeros_like(ts_original)
            ts_ftest = np.zeros_like(ts_original)
            
            for roi in range(num_rois):
                signal = ts_original[:, roi]
                
                # FFT
                n = len(signal)
                fft = np.fft.fft(signal)
                frequencies = np.fft.fftfreq(n, 1/fs)
                
                # Slow-5
                fft_slow5 = fft.copy()
                for i in range(len(frequencies)):
                    freq = abs(frequencies[i])
                    if freq < slow5_min or freq > slow5_max:
                        fft_slow5[i] = 0
                ts_slow5[:, roi] = np.real(np.fft.ifft(fft_slow5))
                
                # Slow-4
                fft_slow4 = fft.copy()
                for i in range(len(frequencies)):
                    freq = abs(frequencies[i])
                    if freq < slow4_min or freq > slow4_max:
                        fft_slow4[i] = 0
                ts_slow4[:, roi] = np.real(np.fft.ifft(fft_slow4))

                # Ftest
                fft_ftest = fft.copy()
                for i in range(len(frequencies)):
                    freq = abs(frequencies[i])
                    if freq < f_test_min or freq > f_test_max:
                        fft_ftest[i] = 0
                ts_ftest[:, roi] = np.real(np.fft.ifft(fft_ftest))

            corr_matrix_slow5 = np.corrcoef(ts_slow5.T)
            corr_matrix_slow4 = np.corrcoef(ts_slow4.T)
            corr_matrix_ftest = np.corrcoef(ts_ftest.T)
            
            correlation_matrices[signal_name]["slow5"].append(corr_matrix_slow5)
            correlation_matrices[signal_name]["slow4"].append(corr_matrix_slow4)
            correlation_matrices[signal_name]["f_test"].append(corr_matrix_ftest)

            
        else:
            print(f"  Not found: {signal_file}")


mean_results_dir = results_dir / "mean_results"
mean_results_dir.mkdir(exist_ok=True)

mean_matrices = {}

for signal_name in signal_names:
    mean_matrices[signal_name] = {}
    
    for band in ["slow4", "slow5", "f_test"]:
        if len(correlation_matrices[signal_name][band]) > 0:
            matrices_array = np.array(correlation_matrices[signal_name][band])
            
            mean_matrix = np.mean(matrices_array, axis=0)
            mean_matrices[signal_name][band] = mean_matrix
            
            with open(mean_results_dir / f"mean_{signal_name}_{band}_correlation_matrix.pkl", 'wb') as f:
                pickle.dump(mean_matrix, f)
            
            plt.figure(figsize=(10, 8))
            im = plt.imshow(mean_matrix, cmap='coolwarm', vmin=-1, vmax=1)
            plt.title(f'Mean {signal_name} {band} Correlation Matrix (n={len(matrices_array)})')
            plt.colorbar(im)
            plt.tight_layout()
            plt.savefig(mean_results_dir / f"mean_{signal_name}_{band}_correlation_matrix.png", dpi=300, bbox_inches='tight')
            plt.close()
            
        else:
            print(f" Not found {signal_name} {band}")



